In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from optimus import Optimus

# Create optimus
op = Optimus()

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [23]:
df = op.read.csv("data/random.csv",header=True, sep=";")

In [2]:
df.table()

## Key Collision

In [3]:
from optimus.ml import keycollision as keyCol

# op.spark.conf.set("spark.sql.shuffle.partitions", "4")
# op.spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1 * 1024 * 1024 * 1024)

keyCol.fingerprint(df, 'STATE').table()

In [4]:
keyCol.fingerprint_cluster(df, 'STATE').table()

STATE_CLUSTER_SIZE 1 (int) not nullable,STATE_CLUSTER 2 (array<string>) nullable,STATE_COUNT 3 (bigint) nullable,STATE_RECOMMENDED 4 (string) nullable
1,['Estado⋅de⋅México'],810,Estado⋅de⋅México
2,"['México⋅D.F.',⋅'Mexico⋅D.F.']",2495,Mexico⋅D.F.
1,['D.F.'],66,D.F.
1,['Distriro⋅Federal'],259,Distriro⋅Federal
3,"['Distrito⋅Federal',⋅'DISTRITO⋅FEDERAL',⋅'distrito⋅federal']",11930,Distrito⋅Federal


In [9]:
keyCol.n_gram_fingerprint(df, 'STATE', 2).table(columns=["STATE","STATE_NGRAM", "STATE_NGRAM_FINGERPRINT"])

STATE_NGRAM STATE_NGRAM_FINGERPRINT


STATE 1 (string) nullable,STATE_NGRAM 2 (array<string>) not nullable,STATE_NGRAM_FINGERPRINT 3 (string) nullable
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],
Distrito⋅Federal,['distritofederal'],


In [10]:
keyCol.n_gram_fingerprint_cluster(df, "STATE" , 2).table()

STATE_NGRAM STATE_NGRAM_FINGERPRINT


STATE_CLUSTER_SIZE 1 (int) not nullable,STATE_CLUSTER 2 (array<string>) nullable,STATE_COUNT 3 (double) nullable,STATE_RECOMMENDED 4 (string) nullable
8,"['Distrito⋅Federal',⋅'México⋅D.F.',⋅'DISTRITO⋅FEDERAL',⋅'Mexico⋅D.F.',⋅'Distr...",15560.0,Mexico⋅D.F.


In [11]:
keyCol.n_gram_fingerprint_cluster(df, "STATE" , 2).to_json()

STATE_NGRAM STATE_NGRAM_FINGERPRINT


[{'STATE_CLUSTER_SIZE': 8,
  'STATE_CLUSTER': ['Distrito Federal',
   'México D.F.',
   'DISTRITO FEDERAL',
   'Mexico D.F.',
   'Distriro Federal',
   'D.F.',
   'Estado de México',
   'distrito federal'],
  'STATE_COUNT': 15560.0,
  'STATE_RECOMMENDED': 'Mexico D.F.'}]

## Distance Cluster

In [12]:
from optimus.ml import distancecluster as dc
dc.levenshtein_matrix(df,"STATE").table()

STATE_LEVENSHTEIN_1 1 (string) nullable,STATE_LEVENSHTEIN_2 2 (string) nullable,STATE_LEVENSHTEIN_DISTANCE 3 (int) nullable
estadodemexico,estadodemexico,0
estadodemexico,mexicodf,10
estadodemexico,df,13
estadodemexico,distrirofederal,11
estadodemexico,distritofederal,11
mexicodf,estadodemexico,10
mexicodf,mexicodf,0
mexicodf,df,6
mexicodf,distrirofederal,12
mexicodf,distritofederal,12


In [13]:
dc.levenshtein_filter(df,"STATE").table()

STATE_FROM 1 (string) nullable,STATE_LEVENSHTEIN_DISTANCE 2 (int) nullable,STATE_TO 3 (string) nullable
estadodemexico,10,mexicodf
df,6,mexicodf
distrirofederal,1,distritofederal
distritofederal,1,distrirofederal
mexicodf,6,df


In [24]:
dc.levenshtein_cluster(df,"STATE").to_json()

[{'STATE_CLUSTER': ['Estado de México'],
  'STATE_CLUSTER_SIZE': 1,
  'STATE_RECOMMENDED': 'Estado de México',
  'STATE_COUNT': 810},
 {'STATE_CLUSTER': ['D.F.'],
  'STATE_CLUSTER_SIZE': 1,
  'STATE_RECOMMENDED': 'D.F.',
  'STATE_COUNT': 66},
 {'STATE_CLUSTER': ['Distriro Federal'],
  'STATE_CLUSTER_SIZE': 1,
  'STATE_RECOMMENDED': 'Distriro Federal',
  'STATE_COUNT': 259},
 {'STATE_CLUSTER': ['Distrito Federal',
   'DISTRITO FEDERAL',
   'distrito federal'],
  'STATE_CLUSTER_SIZE': 3,
  'STATE_RECOMMENDED': 'Distrito Federal',
  'STATE_COUNT': 11930},
 {'STATE_CLUSTER': ['Mexico D.F.', 'México D.F.'],
  'STATE_CLUSTER_SIZE': 2,
  'STATE_RECOMMENDED': 'Mexico D.F.',
  'STATE_COUNT': 2495}]

## Feature Engineering

In [16]:
data = [('Japan', 'Tokyo', 37800000),('USA', 'New York', 19795791),('France', 'Paris', 12341418),
              ('Spain','Madrid',6489162)]
df = op.spark.createDataFrame(data, ["country", "city", "population"])

In [17]:
df.table()

country 1 (string) nullable,city 2 (string) nullable,population 3 (bigint) nullable
Japan,Tokyo,37800000
USA,New⋅York,19795791
France,Paris,12341418
Spain,Madrid,6489162


In [18]:
from optimus.ml import feature as fe

### String to Index

In [19]:
df_sti = fe.string_to_index(df, input_cols=["city", "country"])

In [20]:
df_sti.table()

country 1 (string) nullable,city 2 (string) nullable,population 3 (bigint) nullable,city_INDEX 4 (double) not nullable,country_INDEX 5 (double) not nullable
Japan,Tokyo,37800000,2.0,3.0
USA,New⋅York,19795791,3.0,2.0
France,Paris,12341418,0.0,1.0
Spain,Madrid,6489162,1.0,0.0


### Index to string

In [27]:
# Going back to strings from index
df_its = fe.index_to_string(df_sti, input_cols=["country_INDEX"])

# Show DF with column "county_index" back to string
df_its.table()

country 1 (string) nullable,city 2 (string) nullable,population 3 (bigint) nullable,city_INDEX 4 (double) not nullable,country_INDEX 5 (double) not nullable,country_INDEX_string 6 (string) nullable
Japan,Tokyo,37800000,2.0,3.0,Japan
USA,New⋅York,19795791,3.0,2.0,USA
France,Paris,12341418,0.0,1.0,France
Spain,Madrid,6489162,1.0,0.0,Spain


### One Hot Enconder

In [28]:
# Creating DataFrame
data = [
(0, "a"),
(1, "b"),
(2, "c"),
(3, "a"),
(4, "a"),
(5, "c")
]
df = op.spark.createDataFrame(data,["id", "category"])

# One Hot Encoding
df_ohe = fe.one_hot_encoder(df, input_cols=["id"])

# Show encoded dataframe
df_ohe.table()

id 1 (bigint) nullable,category 2 (string) nullable,id__ENCODED 3 (vector) nullable
0,a,"(5,[0],[1.0])"
1,b,"(5,[1],[1.0])"
2,c,"(5,[2],[1.0])"
3,a,"(5,[3],[1.0])"
4,a,"(5,[4],[1.0])"
5,c,"(5,[],[])"


### Vector assembler

In [29]:
# Import Vectors
from pyspark.ml.linalg import Vectors

# Creating DataFrame
data = [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)]

df = op.spark.createDataFrame(data,["id", "hour", "mobile", "user_features", "clicked"])

# Assemble features
df_va = fe.vector_assembler(df, input_cols=["hour", "mobile", "user_features"])

# Show assembled df
print("Assembled columns 'hour', 'mobile', 'user_features' to vector column 'features'")
df_va.select("features", "clicked").table()

Assembled columns 'hour', 'mobile', 'user_features' to vector column 'features'


features 1 (vector) nullable,clicked 2 (double) nullable
"[18.0,1.0,0.0,10.0,0.5]",1.0


### Normalizer

In [47]:
# Import Vectors
from pyspark.ml.linalg import Vectors

data = [
(0, Vectors.dense([1.0, 0.5, -1.0]),),
(1, Vectors.dense([2.0, 1.0, 1.0]),),
(2, Vectors.dense([4.0, 10.0, 2.0]),)
]

df = op.spark.createDataFrame(data,["id", "features"])

df_norm = fe.normalizer(df, input_cols=["features"], p=2.0)

df_norm.table()

id 1 (bigint) nullable,features 2 (vector) nullable
0,"[1.0,0.5,-1.0]"
1,"[2.0,1.0,1.0]"
2,"[4.0,10.0,2.0]"


id 1 (bigint) nullable,features 2 (vector) nullable,features_NORMALIZED 3 (vector) nullable
0,"[1.0,0.5,-1.0]","[0.6666666666666666,0.3333333333333333,-0.6666666666666666]"
1,"[2.0,1.0,1.0]","[0.8164965809277261,0.4082482904638631,0.4082482904638631]"
2,"[4.0,10.0,2.0]","[0.3651483716701107,0.9128709291752769,0.18257418583505536]"
